<a href="https://colab.research.google.com/github/fclubb/EarthSurfaceProcesses/blob/master/Week4_RiverEvolution/ModellingRiversLandlab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantifying river channel evolution with Landlab
These exercises are based on a project orginally designed by Kelin Whipple at Arizona State University. This notebook was created by Nicole Gasparini at Tulane University and modified by Fiona Clubb at Durham University.

<hr>
<small>For tutorials on learning Landlab, click here: <a href="https://github.com/landlab/landlab/wiki/Tutorials">https://github.com/landlab/landlab/wiki/Tutorials</a></small>
<hr>

In the lecture today we discussed landscape evolution models, how they are set up, and some example applications. In this notebook we are going to implement the simplest model of bedrock erosion, the stream power incision model, within the Landlab landscape evolution model.

The learning outcomes for this notebook are:
* To simulate river profile evolution using Landlab
* To explore the influence of uplift patterns on river profile form
* To explore the influence of erodibility on river profiles

## Working through the practical

To work through this notebook, firstly:
1. **COPY THE NOTEBOOK TO YOUR GOOGLE DRIVE** using the "Copy to Drive" button at the top of the page. 
2. Read through the instructions and execute each code block cell by clicking `Shift and Enter` to see what it does.
3. Do the exercises set out throughout notebook.
4. Save the figures and keep them for the next session.

**What is this notebook?**

This notebook illustrates the evolution of detachment-limited channels in an actively uplifting landscape. The landscape evolves according to the equation:

\begin{equation}
 \frac{d z}{d t} = -K_\text{sp} A^{m_{sp}} S^{n_{sp}} + U
\end{equation}
Here, $K_{sp}$ is the erodibility coefficient on fluvial incision, which is thought to be positively correlated with climate wetness, or storminess (this is hard to quantify) and to be negatively correlated with rock strength (again, rock strength is hard to quantify). $m_{sp}$ and $n_{sp}$ are positive exponents, usually thought to have a ratio, $m_{sp}/n_{sp} \approx 0.5$. $A$ is drainage area and $S$ is the slope of steepest descent ($-\frac{dz}{dx}$) where $x$ is horizontal distance (positive in the downslope direction) and $z$ is elevation. (If slope is negative there is no fluvial erosion.) $U$ is an externally-applied rock uplift field.

The fluvial erosion term is also known as the stream power equation. Before using this notebook you should be familiar with this equation from the lectures and reading. 

For a great overview of the stream power equation, see: 

- Whipple and Tucker, 1999, Dynamics of the stream-power river incision model: Implications for height limits of mountain ranges, landscape response timescales, and research needs, Journal of Geophysical Research.

For some great illustrations of modelling with the stream power equation, see:

- Tucker and Whipple, 2002, Topographic outcomes predicted by stream erosion models: Sensitivity analysis and intermodel comparison, Journal of Geophysical Research.

Helpful background on landscape sensitivity to rock uplift rates and patterns can be found here:

- Kirby and Whipple, 2012, Expression of active tectonics in erosional landscapes, Journal of Structural Geology.

**What will you do?**

In this exercise you will modify the code to get a better understanding of how rock uplift rates and patterns and the erodibility coefficient control fluvial channel form.

Start at the top by reading each block of text and sequentially running each code block (shift - enter OR the play button in each cell).

If you just change one code block and rerun only that code block, only the parts of the code in that code block will be updated. (E.g. if you change parameters but don't reset the code blocks that initialise run time or topography, then these values will not be reset.) 

## Answer these questions first

Answer these questions before running the notebook.

1. What do you think will happen to total relief (defined as the maximum minus the minimum elevation, here area is fixed) and channel slope at steady state if $K_{sp}$ is uniformly increased?
2. What do you think will happen to total relief and channel slope at steady state if $U$ is uniformly increased?
3. How do you think a steady-state landscape with a uniform low rock uplift rate will respond if rock uplift is uniformly increased (relative to a steady base level)? How will channel slopes change through time?

## Installing Landlab
Google Colab doesn't have landlab installed automatically, so we will have to run some code to install it. To do that, run the below code block. This will output a lot of text which you can ignore.
**You should only need to do this once when you start the notebook up - if you restart the runtime, then there is no need to re-run this code block**

In [ ]:
!pip install numpy==1.20.1 &> /dev/null
!pip install landlab &> /dev/null

**Now on to the code...**

First we have to import the parts of Python and Landlab that are needed to run this code. You should not have to change this first code block.

**IF YOU GET AN ERROR HERE, click on "Runtime" in the top menu, and then "Restart runtime". Then try running this cell again.**

In [ ]:
# Code block 1

import copy

import numpy as np
from matplotlib import pyplot as plt

from landlab import RasterModelGrid, imshow_grid
from landlab.components import (
    ChannelProfiler,
    ChiFinder,
    FlowAccumulator,
    SteepnessFinder,
    StreamPowerEroder,
)
from landlab.io import write_esri_ascii

Make a grid and set boundary conditions. This is the same thing we did during our practical on hillslope evolution!

In [ ]:
# Code Block 2

number_of_rows = 50  # number of raster cells in vertical direction (y)
number_of_columns = 100  # number of raster cells in horizontal direction (x)
dxy = 200  # side length of a raster model cell, or resolution [m]

# Below is a raster (square cells) grid, with equal width and height
mg1 = RasterModelGrid((number_of_rows, number_of_columns), dxy)

# Set boundary conditions - only the south side of the grid is open.
# Boolean parameters are sent to function in order of
# east, north, west, south.
mg1.set_closed_boundaries_at_grid_edges(True, True, True, False)

Here we make the initial grid of elevation of zeros with a very small amount of noise to make a more pleasing network. We will make a plot just to show what the initial landscape looks like before we run the model.

In [ ]:
# Code Block 3

np.random.seed(35)  # seed set so our figures are reproducible
mg1_noise = (np.random.rand(mg1.number_of_nodes) / 1000.0
             )  # intial noise on elevation gri

# set up the elevation on the grid
z1 = mg1.add_zeros("topographic__elevation", at="node")
z1 += mg1_noise

# plot the initial topography
imshow_grid(mg1, z1,
            grid_units=("m", "m"),
            var_name="Elevation (m)")
plt.title('Initial topography')

max_elev = np.max(z1)
print("Maximum elevation is ", np.max(z1))

Set parameters related to time.

In [ ]:
# Code Block 4

tmax = 500000  # time for the model to run [yr] (Original value was 500,000 yr)
dt = 1000  # time step [yr] (Original value was 1000 yr)
total_time = 0  # amount of time the landscape has evolved [yr]
# total_time will increase as you keep running the code.

t = np.arange(0, tmax, dt)  # each of the time steps that the code will run

Set parameters for incision and intializing all of the process components that do the work. We also initialize tools for quantifying the landscape.

In [ ]:
# Code Block 5

# Original K_sp value is 0.00001
K_sp = 0.00001  # units vary depending on m_sp and n_sp
m_sp = 0.5  # exponent on drainage area in stream power equation
n_sp = 1.0  # exponent on slope in stream power equation. Divide m_sp by n_sp to get the theta value of the stream power model (starting value = 0.5)

frr = FlowAccumulator(mg1, flow_director='FlowDirectorD8')  # intializing flow routing
spr = StreamPowerEroder(mg1, K_sp=K_sp, m_sp=m_sp, n_sp=n_sp,
                        threshold_sp=0.0)  # initializing stream power incision

theta = m_sp / n_sp
# initialize the component that will calculate channel steepness
sf = SteepnessFinder(mg1, reference_concavity=theta, min_drainage_area=1000.0)
# initialize the component that will calculate the chi index
cf = ChiFinder(mg1,
               min_drainage_area=1000.0,
               reference_concavity=theta,
               use_true_dx=True)

Initialize rock uplift rate. This will need to be changed later.

In [ ]:
# Code Block 6

#  uplift_rate [m/yr] (Original value is 0.0001 m/yr)
# CHANGE THIS VALUE BELOW TO CHANGE THE ROCK UPLIFT RATE.
uplift = 0.0001

# this just sets the uplift rate you have specified above across the whole model grid.
uplift_rate = np.ones(mg1.number_of_nodes) * uplift

Now for the code loop. 

Note that you can rerun Code Block 7 many times, and as long as you don't reset the elevation field (Code Block 3), it will take the already evolved landscape and evolve it even more. If you want to change parameters in other code blocks (e.g. Code Block 5 or 6), you can do that too, and as long as you don't reset the elevation field (Code Block 3) the new parameters will apply on the already evolved topography. 

In [ ]:
# Code Block 7

for ti in t:
    z1[mg1.
       core_nodes] += uplift_rate[mg1.core_nodes] * dt  # uplift the landscape
    frr.run_one_step()  # route flow
    spr.run_one_step(dt)  # fluvial incision
    total_time += dt  # update time keeper
    print(total_time)

Plot the topography. This shows you what our model landscape looks like after we have run the model for your specified runtime using Code Block 7.

In [ ]:
# Code Block 8

imshow_grid(mg1,
            "topographic__elevation",
            grid_units=("m", "m"),
            var_name="Elevation (m)")
title_text = f"$K_{{sp}}$={K_sp}; $time$={total_time} yr; $dx$={dxy} m"
plt.title(title_text)

max_elev = np.max(z1)
print("Maximum elevation is ", np.max(z1))

Plot the slope and area data at each point on the landscape (in log-log space). We will only plot the core nodes because the boundary nodes have slopes that are influenced by the boundary conditions. 

In [ ]:
# Code Block 9

plt.loglog(
    mg1.at_node["drainage_area"][mg1.core_nodes],
    mg1.at_node["topographic__steepest_slope"][mg1.core_nodes],
    "b.",
)
plt.ylabel("Topographic slope")
plt.xlabel("Drainage area (m^2)")
title_text = f"$K_{{sp}}$={K_sp}; $time$={total_time} yr; $dx$={dxy} m"

plt.title(title_text)

It is slightly easier to interpret slope-area data when we look at a single channel, rather than the entire landscape. Below we plot the profile and slope-area data for the three largest channels on the landscape.

In [ ]:
# Code Block 10

# profile the largest channels, set initially to find the mainstem channel in the three biggest watersheds
# you can change the number of watersheds, or choose to plot all the channel segments in the watershed that
# have drainage area below the threshold (here we have set the threshold to the area of a grid cell).
prf = ChannelProfiler(mg1,
                      number_of_watersheds=3,
                      main_channel_only=True,
                      minimum_channel_threshold=dxy**2)
prf.run_one_step()

# plot the elevation as a function of distance upstream
plt.figure(1)
title_text = f"$K_{{sp}}$={K_sp}; $time$={total_time} yr; $dx$={dxy} m"
prf.plot_profiles(xlabel='distance upstream (m)',
                  ylabel='elevation (m)',
                  title=title_text)

# plot the location of the channels in map view
plt.figure(2)
prf.plot_profiles_in_map_view()

# slope-area data in just the profiled channels
plt.figure(3)
for i, outlet_id in enumerate(prf.data_structure):
    for j, segment_id in enumerate(prf.data_structure[outlet_id]):
        if j == 0:
            label = "channel {i}".format(i=i + 1)
        else:
            label = '_nolegend_'
        segment = prf.data_structure[outlet_id][segment_id]
        profile_ids = segment["ids"]
        color = segment["color"]
        plt.loglog(
            mg1.at_node["drainage_area"][profile_ids],
            mg1.at_node["topographic__steepest_slope"][profile_ids],
            '.',
            color=color,
            label=label,
        )

plt.legend(loc="upper left")
plt.xlabel("drainage area (m^2)")
plt.ylabel("channel slope [m/m]")
title_text = f"$K_{{sp}}$={K_sp}; $time$={total_time} yr; $dx$={dxy} m"
plt.title(title_text)

Remember from last week's lecture that slope-area plots are sometimes noisy and difficult to fit a regression to find a correct channel steepness value. We can therefore make plots of chi ($\chi$) against elevation, which allows us to calculate channel steepness across the landscape. First, let's look at what our plots of chi against elevation look like for these same 3 rivers:

In [ ]:
# Code Block 11

# calculate the chi index
cf.calculate_chi()

# chi-elevation plots in the profiled channels
plt.figure(4)

for i, outlet_id in enumerate(prf.data_structure):
    for j, segment_id in enumerate(prf.data_structure[outlet_id]):
        if j == 0:
            label = "channel {i}".format(i=i + 1)
        else:
            label = '_nolegend_'
        segment = prf.data_structure[outlet_id][segment_id]
        profile_ids = segment["ids"]
        color = segment["color"]
        plt.plot(
            mg1.at_node["channel__chi_index"][profile_ids],
            mg1.at_node["topographic__elevation"][profile_ids],
            color=color,
            label=label,
        )

plt.xlabel("chi index (m)")
plt.ylabel("elevation (m)")
plt.legend(loc="lower right")
title_text = f"$K_{{sp}}$={K_sp}; $time$={total_time} yr; $dx$={dxy} m; concavity={theta}"
plt.title(title_text)

We can see that there are convexities in this plot: remember at steady state, there should be a linear relationship between the chi index and elevation. This convexity suggests there is a change in channel steepness as we go upstream. Let's look at channel steepness in the same three largest channels, and also plot steepness index across the grid. Remember we calculate channel steepness by taking the gradient of the line between chi and elevation.

In [ ]:
# Code Block 12

# calculate channel steepness
sf.calculate_steepnesses()

# plots of steepnes vs. distance upstream in the profiled channels
plt.figure(6)

for i, outlet_id in enumerate(prf.data_structure):
    for j, segment_id in enumerate(prf.data_structure[outlet_id]):
        if j == 0:
            label = "channel {i}".format(i=i + 1)
        else:
            label = '_nolegend_'
        segment = prf.data_structure[outlet_id][segment_id]
        profile_ids = segment["ids"]
        distance_upstream = segment["distances"]
        color = segment["color"]
        plt.plot(
            distance_upstream,
            mg1.at_node["channel__steepness_index"][profile_ids],
            'x',
            color=color,
            label=label,
        )

plt.xlabel("distance upstream (m)")
plt.ylabel("Channel steepness")
plt.legend(loc="upper left")
plt.title(
    f"$K_{{sp}}$={K_sp}; $time$={total_time} yr; $dx$={dxy} m; concavity={theta}"
)

# channel steepness map
plt.figure(7)
imshow_grid(
    mg1,
    "channel__steepness_index",
    grid_units=("m", "m"),
    var_name="Channel steepness",
    cmap="jet",
)
title_text = ("$K_{sp}$=" + str(K_sp) + "; $time$=" + str(total_time) +
              "yr; $dx$=" + str(dxy) + "m" + "; concavity=" + str(theta))
plt.title(
    f"$K_{{sp}}$={K_sp}; $time$={total_time} yr; $dx$={dxy} m; concavity={theta}"
)

We can export the model elevations to a DEM, that you can then load into a GIS. If want to do this, edit the `write_file_name` below to be the name that you want to use for the DEM and run the code block.

In [ ]:
# Code Block 13
import os

# Below has the name of the file that data will be written to.
# You need to change the name of the file every time that you want
# to write data, otherwise you will get an error.
# This will write to the directory that you are running the code in.
write_file_name = 'data_file.txt'

# check if the DEM already exists, and if so, remove it to save again
if os.path.exists('./'+write_file_name):
  os.remove('./'+write_file_name)

# Below is writing elevation data in the ESRI ascii format so that it can
# easily be read into Arc GIS or back into Landlab.
write_esri_ascii(write_file_name, mg1, 'topographic__elevation')

Let's write data about the river profiles to a CSV file so that we can open it in Excel if you want to:

In [ ]:
# Code Block 14

import pandas as pd

# loop through the profiles in the channel profiler and save the profile characteristics to a pandas dataframe
prf_df = pd.DataFrame(columns=['channel_no', 'elevation', 'chi', 'channel_steepness', 'drainage_area', 'slope'])
for i, outlet_id in enumerate(prf.data_structure):
    for j, segment_id in enumerate(prf.data_structure[outlet_id]):
        this_df = pd.DataFrame(columns=['channel_no', 'elevation', 'chi', 'channel_steepness', 'drainage_area', 'slope'])
        segment = prf.data_structure[outlet_id][segment_id]
        this_df['elevation'] = mg1.at_node["topographic__elevation"][profile_ids]
        this_df['chi'] = mg1.at_node["channel__chi_index"][profile_ids]
        this_df['channel_steepness'] = mg1.at_node["channel__steepness_index"][profile_ids]
        this_df['drainage_area'] = mg1.at_node["drainage_area"][profile_ids]
        this_df['slope'] = mg1.at_node["topographic__steepest_slope"][profile_ids]
    this_df['channel_no'] = int(i+1)
    prf_df = prf_df.append(this_df)

# Below has the name of the csv file that data will be written to.
# You need to change the name of the file every time that you want
# to write data, otherwise the file will keep overwriting
# This will write to the directory that you are running the code in.
csv_file_name = 'river_profiles.csv'

# check if the CSV already exists, and if so, remove it to save again
if os.path.exists('./'+csv_file_name):
  os.remove('./'+csv_file_name)

# write the dataframe to a csv file
prf_df.to_csv(csv_file_name)

---

## Question...

After running every code block once, has the landscape reached steady state?

Answer: **NO!** How do you know? After you think about this, you are ready to complete the practical.

---

## Practical exercises

Answer the following questions using the code above and below. Code Blocks 8-12 produce different figures that you may find useful. You can use any or all of these different figures to help you with the questions below. (Download or screenshot the figures.) 

1. **Steady state with low uplift rate.** Using the code above, run the landscape to steady state. To do this, run the main evolution loop again - Code Block 7 - and the different plotting blocks WITHOUT running the code blocks above them. (You could also change $tmax$ in Code Block 4 instead). How did you know that the landscape reached steady state? Note the approximate time that it took to reach steady state for your own reference. (This will be useful for later questions.) Include appropriate plots. (If you want to analyze these landscapes outside of Landlab or save for later, make sure you save the elevation data to a text file (Code Block 13) and the river profiles to a CSV file (Code Block 14).)

---------------------

**NOTE, For the rest of the questions you should modify the parameters in Code Blocks 4, 5 and 6. Then run Code Block 7 to add time on to your existing model run or "base" landscape. If you want to start from a flat surface again, then you need to re-run the notebook from Code Block 2.**

--------------------

2. What do you think will happen to this steady-state or "base" landscape if we increased rock uplift rate? How would you expect landscape relief and channel steepness to change? Write down your answers or hypothesis BEFORE you start running any code.

3. What do you think would happen to this base landscape if we instead increased rock erodibility, or $K_{sp}$? How would relief and channel steepness change in this case? Write down your answers or hypothesis BEFORE you start running any code.

4. **Transient landscape responding to an increase in rock uplift.** Use the base landscape and increase rock uplift uniformly by a factor of 4 to 0.0004 m/yr. Make sure you update the rock uplift rate (Code Block 6) and ensure that $tmax$ is 100,000 yrs and $dt$ is 500 yrs (Code Block 4). Once you have changed the parameters in these blocks REMEMBER TO RE-RUN THESE BLOCKS OR THE CODE WILL STILL USE THE OLD PARAMETERS. Run this until the maximum elevation in the grid is ~ 170 m (to re-run for a longer time, run Code Block 7 again) and observe how the landscape gets to this elevation, i.e. plot intermediate steps. What patterns do you see in the supporting plots that show the landscape is transient?

5. **Steady-state landscape with increased rock uplift.** Now keep running with the same parameters until your landscape reaches steady state (i.e. run the time loop, Code Block 7, lots more times: if you want, you can increase $tmax$ and $dt$ to make this run faster). Provide a plot that illustrates that the landscape is in steady state. What aspects of the landscape have changed in comparison with the base landscape from question 1?

6. **Increase erodibility.** Start again with a NEW landscape (re-run from Code Block 2). Make sure rock uplift rate is set to the original value of 0.0001 m/yr (Code Block 6). Run the landscape to steady state with the original $K_{sp}$ value of 0.00001, with $tmax$ of 1,000,000 years and $dt$ of 1,000 years. After you have reached steady state, double $K_{sp}$ to 0.00002 (Code Block 5), then change $tmax$ to 100,000 years and $dt$ to 500 years. Change $tmax$ to 100,000 years, and then run the model again a couple of times. Quantitatively describe how the landscape evolves in response to the increase in erodibility and provide supporting plots. What could cause a uniform increase in erodibility?

7. **Final reflections**. Were your hypotheses correct in questions 2 and 3? It's fine if they weren't - just make sure you understand how we would expect a landscape to respond to changing erodibility and uplift!